## Compilation C en Python

Python est un langage interprété relativement souple. Il n'est pas nécessaire de donner le type des objets que vous manipulez: c'est Python qui, à l'exécution, détermine le type de vos objets pour appeler le code machine correspondant.

Au quotidien, on apprécie notamment de pouvoir manipuler des entiers, des flottants sans se soucier:

- des types,
- des débordements d'entier,
- des débordements de mémoire dans les tableaux,
- des divisions par zéro,
- etc.


In [1]:
print ("Des entiers: {}".format(12 * 3))
print ("Des flottants: {}".format(12 * 3.14))
print ("De très grands entiers: {}".format(12 ** 345))
print ("Des listes trop courtes: {}".format([1, 2][2]))

Des entiers: 36
Des flottants: 37.68
De très grands entiers: 2077446682327378559843444695582704973572786912705232236931705903179519704325276892191015329301807037794598378537132233994613616420526484930777273718077112370160566492728059713895917217042738578562985773221381211423961068296308572143393854703167926779929682604844469621152130457090778409728703018428147734622401526422774317612081074841839507864189781700150115308454681772032


IndexError: list index out of range

Cette souplesse, très appréciée des ingénieurs pour la flexibilité qu'elle offre au quotidien, a un coût: celui de la performance. Le retour au C est souvent nécessaire quand le temps de calcul devient un problème.

La bibliothèque `numpy` que nous avons déjà manipulée propose de revenir au C pour les tableaux de données et fait appel aux fonctions C qui correspondent à l'intuition des opérateurs Python. On a vu par exemple `np.sin(x)` qui déroule une boucle pour calculer un sinus sur tous les éléments d'un tableau.

### Introduction à Cython

Observons le code ci-dessous qui intègre la fonction $f:x \mapsto x^2 - x$ par la méthode des rectangles.

In [ ]:
def f(x):
    return x**2-x

def integrate_f(a, b, N):
    s = 0
    dx = (b-a)/N
    for i in range(N):
        s += f(a+i*dx)
    return s * dx

In [ ]:
%timeit integrate_f(0, 5, 100000)

Cython (http://docs.cython.org/index.html) est une bibliothèque Python qui permet de compiler en C du code natif Python. En pratique, c'est une nouvelle syntaxe qui permet de générer du code C à partir de code Python annoté.

**La bonne nouvelle, c'est que tout code Python non annoté peut être compris par Cython.**

Dans le notebook, on peut utiliser une extension `Cython` qui permet d'automatiser la procédure de compilation et de chargement des fonctions compilées dans Python. Il faut commencer par charger cette extension.

In [ ]:
%load_ext Cython

Puis on préfixe les cellules de code à compiler par la ligne `%%cython`. Il est indispensable que cette instruction soit sur la première ligne de la cellule.

In [ ]:
%%cython

def f(x):
    return x**2-x

def integrate_f(a, b, N):
    s = 0
    dx = (b-a)/N
    for i in range(N):
        s += f(a+i*dx)
    return s * dx

Comparons maintenant les performances.

In [ ]:
%timeit integrate_f(0, 5, 100000)

D'une manière générale, du code Python non annoté permet d'atteindre une amélioration des temps de calcul de 30%.

Mais soyons plus audacieux et annotons nos variables par leur type:
- les variables passées aux fonctions (on ajoute ainsi `double`, `int`, etc.);
- les variables locales à notre fonction (on utilise alors **le mot clé `cdef`**)

In [ ]:
%%cython

def f(double x):
    return x**2-x

def integrate_f(double a, double b, int N):
    cdef int i
    cdef double dx
    s = 0
    dx = (b-a)/N
    for i in range(N):
        s += f(a+i*dx)
    return s * dx

In [ ]:
%timeit integrate_f(0, 5, 100000)

L'écart de performance commence à devenir intéressant.

La commande `%%cython` offre une option `-a` (pour annotate) qui permet de guider nos annotations. Ainsi, dans le résultat qui s'affiche sous la cellule:

- plus une ligne est jaune, plus elle est proche du Python;
- plus une ligne est blanche, plus elle est proche du C.

On peut cliquer sur les lignes jaunes (et blanches) pour « déplier » le code C généré correspondant et comprendre les optimisations qui peuvent encore être faites.

In [ ]:
%%cython -a

def f(double x):
    return x**2-x

def integrate_f(double a, double b, int N):
    cdef int i
    cdef double dx
    s = 0
    dx = (b-a)/N
    for i in range(N):
        s += f(a+i*dx)
    return s * dx

Avec la pratique, ici on comprend que la fonction `f` n'a pas de type de retour. (cf. clic sur la ligne 3)  
En cliquant sur la ligne 2 ou sur la ligne 11, on voit également l'étendue des dégâts (le volume du code C généré).

Il est alors possible de typer la valeur de retour de la fonction `f`. **On remplace alors le mot-clé `def` par le mot clé `cdef`** avant de préciser le type de retour. Le prix à payer pour cette optimisation est que la fonction `f` n'est plus accessible depuis le code Python. Elle peut en revanche être appelée par les autres fonctions dans la cellule Cython.

C'est la raison pour laquelle **il n'est pas possible de typer la valeur de retour de la fonction `integrate_f`**.

In [ ]:
%%cython -a

cdef double f(double x):
    return x**2-x

def integrate_f(double a, double b, int N):
    cdef int i
    cdef double dx
    s = 0
    dx = (b-a)/N
    for i in range(N):
        s += f(a+i*dx)
    return s * dx

In [ ]:
%timeit integrate_f(0, 5, 100000)

<div class="alert alert-warning">
**Exercice:** Il reste une annotation que nous avons oubliée dans le code précédent.  
Complétez l'optimisation que nous avons entamée et comparez les performances.
</div>

<div class="alert alert-warning">
**Exercice:** Comparez les performances entre les différentes étapes de l'optimisation Cython.
</div>

### Appel de fonctions C depuis Python (avancé)

On souhaite appeler la fonction `qsort` de la librairie standard C pour trier des structures Python. Python est certes bien équipé pour trier des listes, mais l'exemple est très illustratif.

La fonction `qsort` est présente dans la librairie standard C, dans `stdlib.h`. On peut y trouver sa signature:
```c
void qsort(void *array, size_t count, size_t size,
           int (*compare)(const void*, const void*))
```

`qsort` fonctionne à base de structures opaques (`void*`): `array` est un tableau à trier qui comprend `count` éléments de taille `size` (en octets). La fonction `compare` prend en entrée deux références (pointeurs) dans le tableau `array` et renvoie un entier négatif si `a < b`, `0` si `a == b` et un entier positif si `a > b`.

On va créer une fonction `py_qsort` qui va trier une liste d'entiers avec la fonction `qsort` en C.

Il va nous falloir:
- créer un tableau C d'entiers de la bonne taille;
- recopier les entiers Python dans le tableau C;
- appeler la fonction `qsort` avec la bonne fonction de comparaison;
- récupérer le tableau trié puis le convertir en tableau Python.

Cython permet de faire appel à des fonctions définies dans des *headers* C grâce à la formule `cdef extern from`, on recopie ensuite ligne par ligne les déclarations de fonctions que l'on souhaite utiliser en Cython.

In [ ]:
%%cython

cdef extern from "stdlib.h":
    void qsort(void *array, size_t count, size_t size,
               int (*compare)(const void*, const void*))
    void *malloc(size_t size)
    void free(void* ptr)
    
# Comparison function
cdef int int_compare(const void* a, const void* b):
    cdef int ia, ib
    ia = (<int*> a)[0] # cast to int -> dereference
    ib = (<int*> b)[0]
    return ia - ib

def py_qsort(list x):
    cdef int *array
    cdef int i, n
    
    # Allocate the C array
    n = len(x)
    array = <int*> malloc(sizeof(int) * n)
    if array == NULL:
        raise MemoryError("Unable to allocate array")
        
    # Fill the C array with the Python integers
    for i in range(n):
        array[i] = x[i]
        
    # qsort the array
    qsort(<void*> array, <size_t> n, sizeof(int), int_compare)
        
    # Convert back to Python and free 
    for i in range(n):
        x[i] = array[i]
    
    free(array)

In [ ]:
from random import shuffle

intlist = list(range(10))
shuffle(intlist)
print (intlist)

In [ ]:
py_qsort(intlist)
print (intlist)

<div class="alert alert-warning">
**Exercice:** Ajouter un mot-clé `reverse` à la fonction `py_qsort` (par défaut à `False`) pour faire un tri dans l'ordre décroissant.
</div>

In [ ]:
%%cython


In [ ]:
py_qsort(intlist)
print (intlist)

shuffle(intlist)
py_qsort(intlist, reverse = True)
print (intlist)  # [9, 8, 7, 6, 5, 4, 3, 2, 1, 0]

<div class="alert alert-warning">
**Exercice:** Ajouter un mot-clé `compare` à la fonction `py_qsort` (par défaut à `None`) pour passer une fonction de comparaison Python.
</div>

In [ ]:
%%cython


In [ ]:
intlist = list(range(-10, 10))
shuffle(intlist)

def cmp(a, b):
    return (abs(a) - abs(b))

py_qsort(intlist, compare = cmp)
print (intlist)  # [0, -1, 1, -2, 2, 3, -3, -4, 4, -5, 5, -6, 6, 7, -7, 8, -8, 9, -9, -10]

shuffle(intlist)
py_qsort(intlist, compare = cmp, reverse = True)
print (intlist)